In [ ]:
import os
from os import listdir
from os.path import isfile, join
from datetime import datetime
import re
import json
import pickle

In [ ]:
from IPython.display import display, Markdown

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.cluster import KMeans
from kneed import KneeLocator

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
import httpimport

with httpimport.remote_repo(
    ['visualizador_dataframes', 'ferramentas_linguagem_natural'],
    'https://bonafe.github.io/CienciaDadosPython/src/componentes/'):
    
    from ferramentas_linguagem_natural import FerramentasLinguagemNatural
    
    from visualizador_dataframes import VisualizadorDataFrames    

In [ ]:
def prefixoData(data):
    return data.strftime("%Y_%m_%d")

In [ ]:
df_instrutores_clusterizado = pd.read_csv("z:/bases_capacita/projeto_capacita/tcc_bonafe/2022_06_30-instrutores_clusterizacao_variaveis_textos.csv", index_col=None, header=0)
df_instrutores_clusterizado.name = "instrutores"

In [ ]:
df_instrutores_clusterizado.sample(5)

# Padronização dos nomes dos Clusters

In [ ]:
de_para_conteudo_programatico = {
    0: "miscelanea",
    1: "auditoria_interna",
    7: "contagil",
    4: "relacionamento_sociedade",
    5: "vigilancia_repressao",
    3: "aduana",
    12: "planejamento",
    11: "credito_tributario",
    10: "projeto_farol",
    6: "tributos_declaracoes",
    8: "cadastros",
    2: "obras",
    9: "plano_gestao"
}

de_para_titulo_acao = {
    12: "vigilancia_repressao",
    0: "relacionamento_sociedade",
    9: "atendimento",
    6: "aduana",
    7: "tributos_declaracoes",
    4: "contagil",
    13: "gestao",
    11: "credito_tributario",
    5: "projeto_farol",
    2: "contagil",
    10: "combate_fraudes",
    3: "pis_cofins",
    8: "imposto_renda",
    1: "cadastros"
}

de_para_nome_curso = {
    14: "vigilancia_repressao",
    2: "atendimento",
    0: "obras",
    5: "relacionamento_sociedade",
    3: "aduana",
    9: "gestao",
    11: "contagil",
    8: "credito_tributario",
    1: "atendimento",
    13: "tributos_declaracoes",
    6: "contagil",
    12: "projeto_farol",
    7: "administracao_tributaria",
    4: "contagil",
    15: "tributos_declaracoes",
    10: "pis_cofins"
}

de_para_objetivo_projeto = {
    10: "direito_creditorio",
    0: "atendimento",
    11: "unidades_rfb",
    6: "gestao",
    1: "aduana",
    3: "credito_tributario",
    5: "gestao",
    9: "aduana",
    7: "contagil",
    4: "tributos_declaracoes",
    2: "projeto_farol",
    12: "vigilancia_repressao",
    8: "gestao"
}

In [ ]:
df_instrutores_clusterizado["cluster_conteudo_programatico_padronizado"] = df_instrutores_clusterizado["cluster_conteudo_programatico"].apply (lambda valor: de_para_conteudo_programatico[valor])
df_instrutores_clusterizado["cluster_titulo_acao_padronizado"] = df_instrutores_clusterizado["cluster_titulo_acao"].apply (lambda valor: de_para_titulo_acao[valor])
df_instrutores_clusterizado["cluster_nome_curso_padronizado"] = df_instrutores_clusterizado["cluster_nome_curso"].apply (lambda valor: de_para_nome_curso[valor])
df_instrutores_clusterizado["cluster_objetivo_projeto_padronizado"] = df_instrutores_clusterizado["cluster_objetivo_projeto"].apply (lambda valor: de_para_objetivo_projeto[valor])

# Clusterização dos Instrutores

In [ ]:
df = df_instrutores_clusterizado.drop(columns = ['id_instrutoria', 'id_turma_execucao', 'id_acao', 'ano_conclusao',
       'ano_evento', 'data_inicio', 'data_fim', 'carga_horaria',
       'objetivo_projeto', 'tipo', 'tipo_competencia', 'titulo_competencia',
       'titulo_acao', 'conteudo_programatico',
       'nome_instrutor_anonimizado', 'nome_rcd_anonimizado',
       'cluster_conteudo_programatico', 'cluster_titulo_acao', 
        'cluster_nome_curso', 'cluster_objetivo_projeto'])

In [ ]:
df = pd.get_dummies(df, prefix="cluster_conteudo_programatico_padronizado", prefix_sep="_", dummy_na=False, columns=["cluster_conteudo_programatico_padronizado"])
df = pd.get_dummies(df, prefix="cluster_titulo_acao_padronizado", prefix_sep="_", dummy_na=False, columns=["cluster_titulo_acao_padronizado"])
df = pd.get_dummies(df, prefix="cluster_nome_curso_padronizado", prefix_sep="_", dummy_na=False, columns=["cluster_nome_curso_padronizado"])
df = pd.get_dummies(df, prefix="cluster_objetivo_projeto_padronizado", prefix_sep="_", dummy_na=False, columns=["cluster_objetivo_projeto_padronizado"])

In [ ]:
df = df.groupby("matricula_siape_anonimizada").sum()

In [ ]:
df.sample(10)

In [ ]:
matriz = df.values

In [ ]:
quantidade_maxima_clusters = 20

sse = []

for quantidade_clusters in range(1, quantidade_maxima_clusters):
            
    #display (f'Gerando cluster {quantidade_clusters} de {quantidade_maxima_clusters}')

    kmeans = KMeans(n_clusters=quantidade_clusters, init='k-means++', max_iter=300, n_init=10, random_state=1)
    kmeans.fit(matriz)
    sse.append(kmeans.inertia_)

kl = KneeLocator(range(1, quantidade_maxima_clusters), sse, curve="convex", direction="decreasing")

numero_ideal_clusters_instrutores = kl.elbow

display (f'Número ideal de clusteres: {numero_ideal_clusters_instrutores}')

fig, ax = plt.subplots(figsize=(10, 4))
fig.suptitle(f'Método do Cotovelo: Clusteres Instrutores', fontsize=20)
plt.grid(b=True)
plt.plot(range(1, quantidade_maxima_clusters), sse)
plt.xticks(range(1, quantidade_maxima_clusters))
plt.xlabel("Número de Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
km = KMeans(n_clusters=5)
km.fit(matriz)
df_instrutores_agrupados = pd.DataFrame([df.index, km.labels_]).T
df_instrutores_agrupados.rename(columns={0:"matricula_siape_anonimizada", 1:"cluster_instrutor"}, inplace=True)

In [ ]:
df_instrutores_agrupados.value_counts("cluster_instrutor")

In [ ]:
df_instrutores_clusterizado_agrupado = pd.merge (
    df_instrutores_clusterizado,  
    df_instrutores_agrupados,  
    how="left", 
    on="matricula_siape_anonimizada")

In [ ]:
df_instrutores_clusterizado_agrupado.value_counts("cluster_instrutor")

In [ ]:
df_instrutores_clusterizado_agrupado.info()

In [ ]:
pasta_destino = f'z:/bases_capacita/projeto_capacita/tcc_bonafe/'
nome_arquivo = f'{pasta_destino}/{prefixoData(datetime.now())}-instrutores_clusterizado.csv'
df_instrutores_clusterizado_agrupado.to_csv (nome_arquivo, index=False)